## Coupled Markov Process on Pitch and Rhythm

## 

In [20]:
# import data and packages
import numpy as np
from fractions import Fraction

#### Using 1st Violin Data Only

In [7]:
# use 1st violin part to predict 16 bars (forwards Markov and backwards?) 
# - include rests in pitch Markov
# - Markov on: pitch, duration (ie onsets), dynamics, articulation, grace notes (duration=0)

#### Using 1st and 2nd Violin Data

In [6]:
# use 2nd violin part to predict 1st violin part

In [4]:
from music21 import *

In [28]:
c = converter.parse(r'C:\Users\User\Documents\MuseScore4\Scores\violins.musicxml')

In [9]:
c.show('musicxml')

In [29]:
pitches, octaves, duration, offset, bars = [], [], [], [], []
data = c.recurse().notes

for tN in data:
    pitches += [[n.name for n in tN.pitches]]
    octaves += [[n.octave for n in tN.pitches]]
    duration += [tN.duration.quarterLength]
    offset += [tN.offset]
    bars += [tN.measureNumber]

In [30]:
octaves[0:10]

[[3], [3], [3], [3], [3], [3], [3], [3], [3], [3]]

In [31]:
m = min(*octaves)[0]
M = max(*octaves)[0]
print(m, M)

3 6


In [32]:
pitches[0:10]

[['G'], ['G'], ['G'], ['G'], ['G'], ['G'], ['G'], ['G'], ['G'], ['G']]

In [33]:
duration[0:10]

[Fraction(1, 3),
 Fraction(1, 3),
 Fraction(1, 3),
 1.0,
 1.0,
 0.5,
 0.5,
 1.0,
 Fraction(1, 3),
 Fraction(1, 3)]

In [34]:
offset[0:10]

[0.0,
 Fraction(1, 3),
 Fraction(2, 3),
 1.0,
 2.0,
 3.0,
 3.5,
 4.0,
 0.0,
 Fraction(1, 3)]

In [35]:
bars[0:10]

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2]

In [36]:
n = data[3]

In [37]:
n.articulations

[]

In [ ]:
# probability of chord, should we even allow this??

In [38]:
pitches_no = np.zeros((len(pitches), 1))
dict_pitches = {'A': 0, 'A#':1, 'B-':1, 'B': 2, 'C-':2, 'B#':3, 'C': 3, 'C#':4, 'D-':4, 'D': 5, 'D#':6, 'E-':6, 'E': 7, 'E#':7, 'F':8, 'F#':9, 'G-':9, 'F##':10, 'G':10, 'G#':11, 'A-':11}

In [40]:
dict_pitches['A#']

1

In [41]:
# build transition matrix for pitches

P = np.zeros([12*(M-m+1), 12*(M-m+1)]) # assume we stay in same range

for i in range(len(pitches)-1):
    for j in range(len(pitches[i])):
        for k in range(len(pitches[i+1])):
            ind1 = (octaves[i][j] - 3) * 12 + dict_pitches[pitches[i][j]]
            ind2 = (octaves[i+1][k] - 3) * 12 + dict_pitches[pitches[i+1][k]]
            P[ind1, ind2] += 1

P = P / np.sum(P, axis=1, keepdims=True)

In [43]:
P

array([[0.25      , 0.        , 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.2       ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.73529412,
        0.02941176],
       [0.        , 0.        , 0.        , ..., 0.        , 0.5       ,
        0.        ]])

In [49]:
stre = c.parts.stream()

In [98]:
# list of note and rest durations, where rests are indicated by strings

rhythm = []

for n in stre.flat.notesAndRests:
    if n.isRest:
        rhythm += [str(float(n.duration.quarterLength))]
    else:
        rhythm += [float(n.duration.quarterLength)]

In [100]:
rhythm_set = set([float(i) for i in rhythm])
mat_ind = list(rhythm_set) + [str(i) for i in rhythm_set]

In [116]:
# build transition matrix for rhythm

R = np.zeros([len(mat_ind), len(mat_ind)])

for i in range(len(rhythm)-1):
    ind1 = mat_ind.index(rhythm[i])
    ind2 = mat_ind.index(rhythm[i+1])
    R[ind1, ind2] += 1

R = R / np.sum(R, axis=1, keepdims=True)

C:\Users\User\AppData\Local\Temp\ipykernel_10024\319010401.py:10: RuntimeWarning: invalid value encountered in divide
  R = R / np.sum(R, axis=1, keepdims=True)


In [117]:
R

array([[0.67251462, 0.32163743, 0.        , 0.        , 0.00584795,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.20717131, 0.45816733, 0.26294821, 0.00796813, 0.01195219,
        0.        , 0.01593625, 0.00398406, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.02390438,
        0.        , 0.        , 0.        , 0.00398406, 0.        ,
        0.        , 0.        , 0.        , 0.00398406, 0.        ,
        0.        ],
       [0.0042735 , 0.2991453 , 0.29059829, 0.        , 0.0042735 ,
        0.        , 0.36324786, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.03846154, 0.        , 0.        , 0.        , 0.        ,
      

In [ ]:
# using their notes

In [16]:
def violin_note(duration, pitch = "C4"):
    # create Note object with pitch and duration
    return note.Note(pitch, quarterLength = duration*(4))

In [17]:
def create_violin_stream(time_sig = None):
    # initialize note stream for violin
    # if time signature is None, no measure splits
    if time_sig == None:
        violPart = stream.Measure()
    else:
        violPart = stream.Stream()
        violPart.timeSignature = meter.TimeSignature(time_sig)
    
    violPart.insert(0, instrument.Violin())
    return violPart

In [18]:
def append_event(duration, original_stream, rest = False, pitch = 'C4'):
    # returns a new_stream obtained by appending a rhythmical event or a rest of given duration to the original_stream
    new_stream = original_stream
    if rest:
        new_stream.append(note.Rest(quarterLength = duration*(4)))
    else:
        new_stream.append(violin_note(duration, pitch))
    return new_stream

In [21]:
def rhythm_from_sequence(durations, time_sig = None, pitch = 'C4', rhythm=None):
    #Generate rhythmic stream from a list of durations. Rests are indicated by specifying a duration as a string
    if rhythm is None:
        # pass an existing stream 'rhythm' to append the durations, otherwise a new one will be created
        rhythm = create_violin_stream(time_sig = time_sig)
    for dur in durations:
        is_rest = False
        if dur != 0:
            if isinstance(dur, str):
                #if duration is given as a string, interpret and rest and turn string into a numerical value
                is_rest = True
                dur = Fraction(dur)
            
            rhythm = append_event(dur, rhythm, rest = is_rest, pitch = pitch) 
    return rhythm

In [25]:
# example
ex = rhythm_from_sequence([3/8, 1/8, '1/2', 1/4], time_sig = '4/4')
append_event(1/4, ex, pitch = 'D4')
ex.show('musicxml')